# IMAGE COLOR CLASSIFIER

In [29]:
import tkinter as tk 
from tkinter import filedialog
from tkinter import messagebox
import pyperclip
from PIL import Image, ImageTk
import numpy as np
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

# GUI setup

In [32]:
root = tk.Tk()
root.title("Image Color Classifier")
root.geometry("450x450")
root.config(bg="light pink")

# Create Menu Bar

In [34]:
menu_bar = tk.Menu(root)
root.config(menu=menu_bar)

# Global variable to store the dominant color

In [37]:
dominant_color = None 
def set_background_color(hex_color):
    root.config(bg=hex_color)

# File Menu

In [41]:
file_menu = tk.Menu(menu_bar, tearoff=0)
menu_bar.add_cascade(label="File", menu=file_menu)
def show_project_info():
    info_window = tk.Toplevel(root)
    info_window.title("Project Information")
    info_window.geometry("400x300")
    info_window.config(bg="lavender blush")

    title = tk.Label(info_window, text="🎨 Image Color Classifier", font=("Arial", 14, "bold"), bg="lavender blush")
    title.pack(pady=10)

    description = (
        "This application allows users to upload an image,\n"
        "analyze its dominant color using KMeans clustering,\n"
        "and classify the color into basic categories.\n\n"
        "🔍 Features:\n"
        "• Image Upload\n"
        "• Dominant Color Detection\n"
        "• Simple Color Classification\n"
        "• User-Friendly UI\n\n"
        "• Then choose edit and refresh to Refresh the page\n\n"
        "🛠 Built with: Tkinter, PIL, NumPy, Scikit-learn"
    )

    info_label = tk.Label(info_window, text=description, justify="left", font=("Arial", 10), bg="lavender blush")
    info_label.pack(pady=5, padx=10)

file_menu.add_command(label="About", command=show_project_info)

# Edit Menu

In [44]:
edit_menu = tk.Menu(menu_bar, tearoff=0)
menu_bar.add_cascade(label="Edit", menu=edit_menu)
def refresh_app():
    image_label.config(image='')
    image_label.image = None
    color_label.config(text="Color: None", fg="black")
    color_box.config(bg="white", text="")
    animate_color_box()  # reset width animation
    
edit_menu.add_command(label="Refresh", command=refresh_app)



# Function for classify color

In [47]:
def classify_color(rgb):
    r, g, b = rgb
    if r < 50 and g < 50 and b < 50:
        return "Black"
    elif r > 200 and g > 200 and b > 200:
        return "White"
    elif abs(r - g) < 10 and abs(r - b) < 10 and abs(g - b) < 10:
        return "Black and White"
    elif r > 200 and g > 180 and b < 100:
        return "Yellow"
    elif r > g and r > b:
        return "Red"
    elif g > r and g > b:
        return "Green"
    elif b > r and b > g:
        return "Blue"
    else:
        return "Unknown"

# Get dominant color using KMeans

In [50]:
def get_dominant_color(image, k=1):
    image = image.resize((100, 100))
    img_array = np.array(image)
    img_array = img_array.reshape((-1, 3))
    kmeans = KMeans(n_clusters=k, n_init=10)
    kmeans.fit(img_array)
    dominant = kmeans.cluster_centers_[0].astype(int)
    return tuple(dominant)

# Upload and Process Image

In [53]:
def upload_image():
    global dominant_color  # Ensure dominant_color is accessible here
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png *.gif")])
    if file_path:
        img = Image.open(file_path)
        resized_img = img.resize((300, 200))
        tk_img = ImageTk.PhotoImage(resized_img)

        # Show image
        image_label.config(image=tk_img)
        image_label.image = tk_img

        # Get dominant color
        dominant_color = get_dominant_color(img)
        hex_color = f'#{dominant_color[0]:02x}{dominant_color[1]:02x}{dominant_color[2]:02x}'
        color_box.config(bg=hex_color)

        # Classify the color name
        color_name = classify_color(dominant_color)
        color_label.config(text=f"Color: {color_name}\nRGB: {dominant_color}\nHEX: {hex_color}")

        set_background_color(hex_color)


# Function to copy RGB to clipboard
def copy_rgb():
    global dominant_color  # Ensure the dominant color is accessible
    if dominant_color is not None:
        rgb_value = str(dominant_color)  # Use the dominant color variable
        pyperclip.copy(rgb_value)
        messagebox.showinfo("Copied", f"RGB: {rgb_value} copied to clipboard!")
    else:
        messagebox.showwarning("No Color", "No color detected. Please upload an image first.")

# Function to copy HEX to clipboard
def copy_hex():
    global dominant_color  # Ensure the dominant color is accessible
    if dominant_color is not None:
        hex_value = f'#{dominant_color[0]:02x}{dominant_color[1]:02x}{dominant_color[2]:02x}'
        pyperclip.copy(hex_value)
        messagebox.showinfo("Copied", f"HEX: {hex_value} copied to clipboard!")
    else:
        messagebox.showwarning("No Color", "No color detected. Please upload an image first.")



# Main frame for UI components
main_frame = tk.Frame(root, bg="Lavender")
main_frame.pack(expand=True)


# Upload button
upload_btn = tk.Button(main_frame, text="Upload Image", font=("Arial", 12),command=upload_image)
upload_btn.pack(pady=20)

# Image preview
image_label = tk.Label(main_frame, bg="white", width=300, height=200)
image_label.pack(pady=10)

# Color box
color_box = tk.Label(main_frame, width=20, height=2, bg="white", relief="ridge")
color_box.pack(pady=5)


# Color label
color_label = tk.Label(main_frame, text="🎨 Color: None", font=("Arial", 12), bg="#f5f5f5")
color_label.pack(pady=10)

# Copy RGB button
rgb_button = tk.Button(main_frame, text="Copy RGB", font=("Arial", 12), command=copy_rgb)
rgb_button.pack(pady=10)

# Copy HEX button
hex_button = tk.Button(main_frame, text="Copy HEX", font=("Arial", 12), command=copy_hex)
hex_button.pack(pady=10)


root.mainloop()